![Shopping trolley in front of a laptop](./iStock-1249219777.jpg)

It's simple to buy any product with a click and have it delivered to your door. Online shopping has been rapidly evolving over the last few years, making our lives easier. But behind the scenes, e-commerce companies face a complex challenge that needs to be addressed. 

Uncertainty plays a big role in how the supply chains plan and organize their operations to ensure that the products are delivered on time. These uncertainties can lead to challenges such as stockouts, delayed deliveries, and increased operational costs.

You work for the Sales & Operations Planning (S&OP) team at a multinational e-commerce company. They need your help to assist in planning for the upcoming end-of-the-year sales. They want to use your insights to plan for promotional opportunities and manage their inventory. This effort is to ensure they have the right products in stock when needed and ensure their customers are satisfied with the prompt delivery to their doorstep.


## The Data

You are provided with a sales dataset to use. A summary and preview are provided below.

# Online Retail.csv

| Column     | Description              |
|------------|--------------------------|
| `'InvoiceNo'` | A 6-digit number uniquely assigned to each transaction |
| `'StockCode'` | A 5-digit number uniquely assigned to each distinct product |
| `'Description'` | The product name |
| `'Quantity'` | The quantity of each product (item) per transaction |
| `'UnitPrice'` | Product price per unit |
| `'CustomerID'` | A 5-digit number uniquely assigned to each customer |
| `'Country'` | The name of the country where each customer resides |
| `'InvoiceDate'` | The day and time when each transaction was generated `"MM/DD/YYYY"` |
| `'Year'` | The year when each transaction was generated |
| `'Month'` | The month when each transaction was generated |
| `'Week'` | The week when each transaction was generated (`1`-`52`) |
| `'Day'` | The day of the month when each transaction was generated (`1`-`31`) |
| `'DayOfWeek'` | The day of the weeke when each transaction was generated <br>(`0` = Monday, `6` = Sunday) |

In [6]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import col, dayofmonth, month, year,  to_date, to_timestamp, weekofyear, dayofweek
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Initialize Spark session
my_spark = SparkSession.builder.appName("SalesForecast").getOrCreate()

# Importing sales data
sales_data = my_spark.read.csv(
    "Online Retail.csv", header=True, inferSchema=True, sep=",")

# Convert InvoiceDate to datetime 
sales_data = sales_data.withColumn("InvoiceDate", to_date(
    to_timestamp(col("InvoiceDate"), "d/M/yyyy H:mm")))

In [7]:
# Aggregate data into daily intervals
daily_sales_data = sales_data.groupBy(
    "Country", "StockCode", "InvoiceDate", "Year", "Month", "Day", "Week", "DayOfWeek"
).agg(
    {"Quantity": "sum", "UnitPrice": "avg"}
).withColumnRenamed(
    "sum(Quantity)", "Quantity"
)

# Split date for train-test split
split_date_train_test = "2011-09-25"

# Create train and test datasets
train_data = daily_sales_data.filter(col("InvoiceDate") <= split_date_train_test)
test_data = daily_sales_data.filter(col("InvoiceDate") > split_date_train_test)

# Convert train data to pandas
pd_daily_train_data = train_data.toPandas()

# Create indexers for categorical columns
indexers = [
    StringIndexer(inputCol=col_name, outputCol=f"{col_name}Index").setHandleInvalid("keep")
    for col_name in ["Country", "StockCode"]
]

# Combine features
assembler = VectorAssembler(inputCols=["CountryIndex", "StockCodeIndex", "Month", "Year", "DayOfWeek", "Day", "Week"], outputCol="features")

# Initialize Random Forest model
rf = RandomForestRegressor(featuresCol="features", labelCol="Quantity", maxBins=4000)

# Create a pipeline
pipeline = Pipeline(stages=indexers + [assembler, rf])

# Train the model
model = pipeline.fit(train_data)

# Get test predictions
test_predictions = model.transform(test_data).withColumn("prediction", col("prediction").cast("double"))

# Initialize evaluator
mae_evaluator = RegressionEvaluator(labelCol="Quantity", predictionCol="prediction", metricName="mae")

# Obtain MAE
mae = mae_evaluator.evaluate(test_predictions)

# Get units sold during week 39 of 2011
quantity_sold_w39 = int(test_predictions.filter((col("Year") == 2011) & (col("Week") == 39)).agg({"prediction": "sum"}).collect()[0][0])

# Stop Spark session
my_spark.stop()

24/04/12 09:40:33 WARN DAGScheduler: Broadcasting large task binary with size 1730.8 KiB


24/04/12 09:40:35 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
